In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train = torchvision.datasets.FashionMNIST('', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

test = torchvision.datasets.FashionMNIST('', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True, num_workers=2)

testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False, num_workers=2)

100.0%

Extracting FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/raw


111.0%

Extracting FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/raw


100.0%

Extracting FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/raw


159.1%

Extracting FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
trainset_shape = trainset.dataset.train_data.shape
testset_shape = testset.dataset.test_data.shape

print(trainset_shape, testset_shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


/home/scarlet/Projects/PyEnvs/DSPyTorchEnv/DSPyTEnv/lib/python3.7/site-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/scarlet/Projects/PyEnvs/DSPyTorchEnv/DSPyTEnv/lib/python3.7/site-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")


In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1) 

In [5]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.005)

In [7]:
for epoch in range(5): 
    for data in trainset:  
        X, y = data  
        net.zero_grad()  
        output = net(X.view(-1,784))  
        loss = loss_criterion(output, y)  
        loss.backward()  
        optimizer.step()  
    print(loss)  

tensor(0.4899, grad_fn=<NllLossBackward>)
tensor(0.9561, grad_fn=<NllLossBackward>)
tensor(0.3555, grad_fn=<NllLossBackward>)
tensor(0.2254, grad_fn=<NllLossBackward>)
tensor(0.4833, grad_fn=<NllLossBackward>)


In [8]:
correct = 0
total = 0

# incorr = []
incorr_X = []
incorr_y = []
otpt = []
torch_argmax_i = []
y_indx = []
i_trch_argmx = []
idx_y = []

# y_indx[4] torch_argmax_i[4] 

corr_y_indx = []
corr_torch_argmax_i = []

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1,784))
        
        for idx, i in enumerate(output):
               if torch.argmax(i) == y[idx]:
                correct += 1
                corr_y_indx.append(y[idx])
                corr_torch_argmax_i.append(torch.argmax(i))

               else:
#                 incorr.append([X, y, output, torch.argmax(i), y[idx]])
                     incorr_X.append(X) 
                     incorr_y.append(y)
                     otpt.append(output)
                     torch_argmax_i.append(torch.argmax(i))
                     y_indx.append(y[idx])
                     i_trch_argmx.append(i)
                     idx_y.append(idx)
               total += 1

print("Accuracy: ", round(correct/total, 2))

Accuracy:  0.84


In [ ]:
# Predicted Label vs Ground Truth

In [9]:
for i in range(10):
    print(corr_torch_argmax_i[i])
    print(corr_y_indx[i])
    print()

tensor(9)
tensor(9)

tensor(2)
tensor(2)

tensor(1)
tensor(1)

tensor(1)
tensor(1)

tensor(6)
tensor(6)

tensor(1)
tensor(1)

tensor(4)
tensor(4)

tensor(6)
tensor(6)

tensor(5)
tensor(5)

tensor(7)
tensor(7)



In [ ]:
# Predicted Label vs Ground Truth

In [10]:
for i in range(10):
    print(torch_argmax_i[i])
    print(y_indx[i])
    print()

tensor(5)
tensor(7)

tensor(2)
tensor(4)

tensor(6)
tensor(2)

tensor(5)
tensor(9)

tensor(2)
tensor(4)

tensor(4)
tensor(3)

tensor(0)
tensor(6)

tensor(6)
tensor(3)

tensor(5)
tensor(7)

tensor(6)
tensor(2)

